Import LDCs greenfield FDI info for years 2017-2018. List of LDCs taken from UNCTAD (2017, 2018). Greenfield obtained from FT fDi Markets database and captures all transactions over USD 500,000. "Source country" variable is the country of the ultimate parent, confirmed via email with Juanjo Lopez on 29 October, 2024. 2017_2018_greenfield_data_fdi_markets_original.csv saved as "main_dataset_greenfield.csv" and all variables and all country names converted to snake_case to faciliate data wrangling.

In [97]:
import pandas as pd

# Load data from fDi Markets
fdi_data = "./raw_data/2017_2018_greenfield_data_fdi_markets_original.csv"
df = pd.read_csv(fdi_data)


# Define function to convert text to snake_case
def to_snake_case(s):
    return s.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()


# Convert variable names (i.e., column headers) to snake_case for data wrangling
df.columns = [to_snake_case(col) for col in df.columns]

# Convert values in "destination_country" and "source_country" to snake_case
df["destination_country"] = df["destination_country"].apply(to_snake_case)
df["source_country"] = df["source_country"].apply(to_snake_case)

# Save modified DataFrame to new CSV file
output_filepath = "./processed_data/greenfield_main_dataset.csv"
df.to_csv(output_filepath, index=False)

print(f"File saved to {output_filepath}")

File saved to ./processed_data/greenfield_main_dataset.csv


Check for instances where source_country == destination_country (i.e., instances of non-FDI). Number should be zero based on description of data from Financial Times fDi.

In [98]:
import pandas as pd

# Load the processed greenfield dataset
filepath = "./processed_data/greenfield_main_dataset.csv"
df = pd.read_csv(filepath)

# Check for existence of columns in question
if "source_country" in df.columns and "destination_country" in df.columns:

    # Filter rows where source_country == destination_country
    non_fdi_rows = df[df["source_country"] == df["destination_country"]]

    # Get the count
    count = len(non_fdi_rows)

    print(f"Number of instances where source_country == destination_country: {count}")

else:
    print("The dataset does not contain 'source_country' or 'destination_country'")

Number of instances where source_country == destination_country: 0


Create spreadsheet of unique destination countries for data auditing/validation.

In [99]:
import pandas as pd

# Load processed greenfield dataset
filepath = "./processed_data/greenfield_main_dataset.csv"
df = pd.read_csv(filepath)

# Get unique country names and count of each
unique_destination_countries = df["destination_country"].value_counts().reset_index()

# Rename columns
unique_destination_countries.columns = ["country", "count"]

# Sort new df alphabetically
unique_destination_countries = unique_destination_countries.sort_values(
    by="country"
).reset_index(drop=True)

# Save to CSV
output_filepath = "./audit_data/unique_destinationa_countries.csv"
unique_destination_countries.to_csv(output_filepath, index=False)

print(f"File saved to {output_filepath}")

# Display results
unique_destination_countries

File saved to ./audit_data/unique_destinationa_countries.csv


,country,count
0,afghanistan,3
1,angola,11
2,bangladesh,47
3,benin,8
4,bhutan,6
5,burkina_faso,8
6,burundi,4
7,cambodia,92
8,central_african_republic,2
9,chad,2


Compare list of destination countries with UNCTAD 2017-2018 LDCs list for data audit/validation purposes.

In [100]:
import pandas as pd

# Load both spreadsheets into DataFrames
destination_countries_df = pd.read_csv("./audit_data/unique_destinationa_countries.csv")
ldcs_df = pd.read_csv("./variable_data/ldcs_list_2017_2018.csv")


# Convert country names to snake_case
def to_snake_case(s):
    return s.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()


destination_countries_df["country"] = destination_countries_df["country"].apply(
    to_snake_case
)
ldcs_df["country"] = ldcs_df["country"].apply(to_snake_case)

# Convert pandas Series objects into sets for comparison
destination_countries_set = set(destination_countries_df["country"])
ldcs_set = set(ldcs_df["country"])

# Create new sets of missing countries
missing_in_destination_countries_list = sorted(ldcs_set - destination_countries_set)
missing_in_ldcs_list = sorted(destination_countries_set - ldcs_set)

# Display results
print("LDCs not appearing as desintation countries in greenfield_main_dataset.csv:")
for country in missing_in_destination_countries_list:
    print(country)
print(
    "Countries appearing as destination countries in greenfield_main_dataset.csv but not appearing in LDCs list:"
)
for country in missing_in_ldcs_list:
    print(country)

LDCs not appearing as desintation countries in greenfield_main_dataset.csv:
comoros
eritrea
guinea_bissau
kiribati
sao_tome_and_principe
solomon_islands
south_sudan
tuvalu
vanuatu
yemen
Countries appearing as destination countries in greenfield_main_dataset.csv but not appearing in LDCs list:


LDCs not appearing in greenfield FDI are consistent with those for which there was no data in fDi Markets for those years.

Create spreadsheet of unique source countries for data auditing/validation.

In [101]:
import pandas as pd

# Load processed greenfield dataset
filepath = "./processed_data/greenfield_main_dataset.csv"
df = pd.read_csv(filepath)

# Get unique source country names and count of each
unique_source_countries = df["source_country"].value_counts().reset_index()

# Rename columns
unique_source_countries.columns = ["country", "count"]

# Sort new df alphabetically
unique_source_countries = unique_source_countries.sort_values(by="country").reset_index(
    drop=True
)

# Save to CSV
output_filepath = "./audit_data/unique_source_countries.csv"
unique_source_countries.to_csv(output_filepath, index=False)

print(f"File saved to {output_filepath}")

# Display results
unique_source_countries

File saved to ./audit_data/unique_source_countries.csv


,country,count
0,australia,4
1,austria,2
2,azerbaijan,1
3,barbados,1
4,belgium,6
...,...,...
58,uae,26
59,ukraine,4
60,united_kingdom,28
61,united_states,49


Compare list of source countries with UNCTAD 2017-2018 developed country list for data audit/validation purposes.

In [102]:
import pandas as pd

# Load both spreadsheets into DataFrames
source_countries_df = pd.read_csv("./audit_data/unique_source_countries.csv")
developed_countries_df = pd.read_csv(
    "./variable_data/developed_countries_2017_2018.csv"
)


# Convert country names to snake_case
def to_snake_case(s):
    return s.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()


source_countries_df["country"] = source_countries_df["country"].apply(to_snake_case)
developed_countries_df["country"] = developed_countries_df["country"].apply(
    to_snake_case
)

# Convert pandas Series objects into sets for comparison
source_countries_set = set(source_countries_df["country"])
developed_countries_set = set(developed_countries_df["country"])

# Create new set of missing countries
missing_in_source_countries = developed_countries_set - source_countries_set

# Display results
print(
    "Developed countries not appearing as source countries in greenfield_main_dataset.csv:"
)
for country in sorted(missing_in_source_countries):
    print(country)

Developed countries not appearing as source countries in greenfield_main_dataset.csv:
andorra
bermuda
bulgaria
czechia
estonia
faroe_islands
gibraltar
greece
greenland
holy_see
hungary
iceland
ireland
latvia
lithuania
luxembourg
malta
poland
romania
saint_pierre_and_miquelon
san_marino
slovakia
slovenia
sweden


Developed countries not appearing in greenfield_main_dataset were checked manually against unique_source_countries.csv to ensure no matches were missed due to spelling or punctuation issues.

Compare list of countries in tax_havens.csv with countries appearing in source_country column and identify any tax_haven countries not appearing as source countries.

In [103]:
import pandas as pd

# Load both datasets
source_countries_df = pd.read_csv("./audit_data/unique_source_countries.csv")
tax_havens_df = pd.read_csv("./variable_data/tax_havens.csv")


# Covert country names to snake_case
def to_snake_case(s):
    return s.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()


source_countries_df["countries"] = source_countries_df["country"].apply(to_snake_case)
tax_havens_df["country"] = tax_havens_df["country"].apply(to_snake_case)

# Convert Pandas Series objects into sets for comparison
source_countries_set = set(source_countries_df["country"])
tax_havens_set = set(tax_havens_df["country"])

# Create set of missing countries
missing_tax_havens_set = tax_havens_set - source_countries_set

# Display results
print(
    "Tax haven countries not appearing as source countries in greenfield_main_dataset.csv:"
)

for country in sorted(missing_tax_havens_set):
    print(country)

Tax haven countries not appearing as source countries in greenfield_main_dataset.csv:
andorra
anguilla
antigua_and_barbuda
aruba
bahamas
bahrain
bermuda
british_virgin_islands
cayman_islands
gibraltar
grenada
guernsey
ireland
isle_of_man
jersey
lebanon
liechtenstein
luxembourg
macau
malta
marshall_islands
monaco
netherlands_antilles
panama
puerto_rico
samoa
seychelles
st_kitts_and_nevis
st_lucia
st_vincent_and_grenadines
turks_and_caicos
vanuatu


Tax haven countries not appearing in greenfield_main_dataset were checked manually against unique_source_countries.csv to ensure no matches were missed due to spelling or punctuation issues.

Create source_is_th variable. Value will be True if source_country appears in list of tax_havens and False otherwise. Tax havens list taken from footnote 10 on page 1506 of Tørsløv, T., Wier, L., & Zucman, G. (2023). The missing profits of nations. The Review of Economic Studies, 90(3), 1499-1534.

In [104]:
import pandas as pd

# Load the spreadsheets into DataFrames

greenfield_df = pd.read_csv("./processed_data/greenfield_main_dataset.csv")
tax_havens_df = pd.read_csv("./variable_data/tax_havens.csv")

# Create set of tax havens
tax_havens_set = set(tax_havens_df["country"])

# Add "source_is_th" variable to greenfield_df
greenfield_df["source_is_th"] = greenfield_df["source_country"].isin(tax_havens_set)

greenfield_df.to_csv("./processed_data/greenfield_main_dataset.csv", index=False)

print('greenfield_main_dataset.csv updated with "source_is_th" variable.')

# Output the number of observations where "source_is_th" is True to console
true_count = greenfield_df["source_is_th"].sum()
print(f'Number of observations where "source_is_th" is True: {true_count}')

greenfield_main_dataset.csv updated with "source_is_th" variable.
Number of observations where "source_is_th" is True: 96


Create destination_is_th variable. Value will be True if destination_country appears in list of tax_havens and False otherwise. Tax havens list taken from footnote 10 on page 1506 of Tørsløv, T., Wier, L., & Zucman, G. (2023). The missing profits of nations. The Review of Economic Studies, 90(3), 1499-1534.

In [105]:
import pandas as pd

# Load the spreadsheets into DataFrames

greenfield_df = pd.read_csv("./processed_data/greenfield_main_dataset.csv")
tax_havens_df = pd.read_csv("./variable_data/tax_havens.csv")

# Create set of tax havens
tax_havens_set = set(tax_havens_df["country"])

# Add "destination_is_th" variable to greenfield_df
greenfield_df["destination_is_th"] = greenfield_df["destination_country"].isin(
    tax_havens_set
)

greenfield_df.to_csv("./processed_data/greenfield_main_dataset.csv", index=False)

print('greenfield_main_dataset.csv updated with "destination_is_th" variable.')

# Output the number of observations where "destination_is_th" is True to console
true_count = greenfield_df["destination_is_th"].sum()
print(f'Number of observations where "destination_is_th" is True: {true_count}')

greenfield_main_dataset.csv updated with "destination_is_th" variable.
Number of observations where "destination_is_th" is True: 0


Create dataset comprising all observations where source_country is a tax haven and manually check accuracy as true origin of FDI.

In [106]:
import pandas as pd

# Load the datasets
greenfield_df = pd.read_csv("./processed_data/greenfield_main_dataset.csv")

# Filter for those instances were source_is_th == True
greenfield_source_is_th_df = greenfield_df[greenfield_df["source_is_th"] == True]

# Select specific columns
selected_columns = [
    "parent_company",
    "company_profile",
    "website",
    "source_country",
    "sector",
    "sub_sector",
    "cluster",
    "activity",
]

# Retain only unique rows based on 'parent_company' and selected columns
unique_parent_company_df = greenfield_source_is_th_df[selected_columns].drop_duplicates(
    subset=["parent_company"]
)

# Convert new dataframe to CSV for manual checking
unique_parent_company_df.to_csv(
    "./audit_data/greenfield_source_is_th_unique.csv", index=False
)

num_observations = unique_parent_company_df.shape[0]

print("Filtered data saved to greenfield_source_is_th_unique.csv")
print(f"Number of unique parent companies in filtered data: {num_observations}")

Filtered data saved to greenfield_source_is_th_unique.csv
Number of unique parent companies in filtered data: 63


Manually check all companies to determine accuracy of parent country. Results/comments saved to "./audit_data/source_country_th_man_flagged.csv". Companies with source countries identified as tax havens were checked manually on S&P Capital IQ, Orbis, and on company websites. In instances where the stated source country seemed to be either incorrect, or seemed questionable, source_country_questionable is flagged as TRUE.

Create new dataset containing only parent companies where source country is questionable.

In [107]:
import pandas as pd

# Load the CSV file
filepath = "./audit_data/source_country_th_man_flagged.csv"
data = pd.read_csv(filepath)

# Filter rows where 'source_country_questionable' is True
filtered_data = data[data["source_country_questionable"] == True]

# Save the filtered data to a new CSV file
filtered_filepath = "./audit_data/greenfield_source_questionable.csv"
filtered_data.to_csv(filtered_filepath, index=False)

print(f"Fitlered data saved to {filtered_filepath}")

Fitlered data saved to ./audit_data/greenfield_source_questionable.csv


Use flagged data to create a variable source_questionable in the greenfield_main_dataset.csv. Value is True if the source country has been flagged as questionable in the greenfield_source_questionable_csv and false otherwise.

In [108]:
import pandas as pd

# Load the datasets
main_dataset_filepath = "./processed_data/greenfield_main_dataset.csv"
questionable_source_filepath = "./audit_data/greenfield_source_questionable.csv"

main_dataset = pd.read_csv(main_dataset_filepath)
questionable_source = pd.read_csv(questionable_source_filepath)

# Extract list of parent companies from questionable source dataset
questionable_parent_companies = questionable_source["parent_company"].unique()

# Create the "source_questionable" column in main dataset
main_dataset["source_questionable"] = main_dataset["parent_company"].isin(
    questionable_parent_companies
)

# Save the updated dataset to CSV file
main_dataset.to_csv(main_dataset_filepath, index=False)

print(f"Updated dataset saved to {main_dataset_filepath}")

true_count = main_dataset["source_questionable"].sum()
print(f"Number of observations with source_questionable = True: {true_count}")

Updated dataset saved to ./processed_data/greenfield_main_dataset.csv
Number of observations with source_questionable = True: 22


9 observations flagged in the greenfield dataset where the true source country is questionable after manual checking of all instances where "source_country" is a tax haven (see above re: list of tax haven countries).

Create "developed" variable in greenfield_main_dataset.csv. Variable takes value of True if source country appears in list of developed countries and False otherwise. Also created "emne" variable that takes the inverse value of the "developed" variable.

In [109]:
import pandas as pd

# Load spreadsheets into DataFrames

greenfield_filepath = "./processed_data/greenfield_main_dataset.csv"
developed_countries_filepath = "./variable_data/developed_countries_2017_2018.csv"

greenfield_df = pd.read_csv(greenfield_filepath)
developed_countries_df = pd.read_csv(developed_countries_filepath)

# Create set of developed countries
developed_countries_set = set(developed_countries_df["country"])

# Add "developed" variable to greenfield_df
greenfield_df["developed"] = greenfield_df["source_country"].isin(
    developed_countries_set
)

# Add "emne" variable to greenfield_df
greenfield_df["emne"] = ~greenfield_df["source_country"].isin(developed_countries_set)

# Save the updated DataFrame as greenfield_main_dataset.csv
greenfield_df.to_csv(greenfield_filepath, index=False)

print(f'greenfield_main_dataset.csv updated with "developed" and "emne" variables.')

greenfield_main_dataset.csv updated with "developed" and "emne" variables.


Create "bordering_country" variable. Variable is True if source countries shares a border with destination country and False otherwise. Bordering countries checked from CIA World Factbook https://www.cia.gov/the-world-factbook/ 

In [110]:
import pandas as pd

# Load spreadsheets into DataFrames
greenfield_filepath = "./processed_data/greenfield_main_dataset.csv"
bordering_countries_filepath = "./variable_data/bordering_countries_2017_2018.csv"

greenfield_df = pd.read_csv(greenfield_filepath)
bordering_countries_df = pd.read_csv(bordering_countries_filepath)

# Create dictionary of bordering countries
bordering_countries_dict = bordering_countries_df.set_index("ldc")[
    "bordering_countries"
].to_dict()


# Function to check if the source country is in the list of bordering countries
def is_bordering(source_country, destination_country):
    bordering_countries = bordering_countries_dict.get(destination_country, "")
    if pd.isna(bordering_countries) or not bordering_countries:
        return False
    bordering_list = bordering_countries.split(", ")
    return source_country in bordering_list


# Apply the is_bordering function to create the new "bordering_country" variable in greenfield_df
greenfield_df["bordering_country"] = greenfield_df.apply(
    lambda row: is_bordering(row["source_country"], row["destination_country"]), axis=1
)

# Save the updated DataFrame to a new CSV file
greenfield_df.to_csv(greenfield_filepath, index=False)

print(
    f'The new variable "bordering country" has been added and the updated file has been saved.'
)

The new variable "bordering country" has been added and the updated file has been saved.


Create new "industry" variable by joining "sector", "sub_sector", and "activity". Create list/spreadsheet of all unique values in "industry" column (and their count) from greenfield_main_dataset.csv.

In [111]:
import pandas as pd

# Load processed greenfield dataset
filepath = "./processed_data/greenfield_main_dataset.csv"
df = pd.read_csv(filepath)


# Define function to convert strings to snake_case
def to_snake_case(s):
    s = (
        s.replace(" ", "_")
        .replace("'", "_")
        .replace("&", "and")
        .replace(",", "")
        .replace("/", "")
        .replace("-", "_")
        .replace(",", "")
        .replace("(", "")
        .replace(")", "")
    )
    return s.lower()


# Create a new 'industry' column by joining 'sector', 'sub-sector', and 'activity'
df["industry"] = (
    df[["sector", "sub_sector", "activity"]]
    .fillna("")
    .apply(lambda x: " ".join(x), axis=1)
)

# Convert the new 'industry' column to snake_case
df["industry"] = df["industry"].apply(to_snake_case)

# Save the modified DataFrame to CSV
df.to_csv(filepath, index=False)

# Calculate the count of unique values in 'industry' column
industry_counts = df["industry"].value_counts().reset_index()
industry_counts.columns = ["industry", "count"]

# Sort alphabetically
industry_counts.sort_values(by="industry", inplace=True)

# Save to CSV
industry_counts_filepath = "./audit_data/industry_counts.csv"
industry_counts.to_csv(industry_counts_filepath, index=False)

print(
    f"Updated dataset has been saved to {filepath}.\n\
        Industry counts has been saved to {industry_counts_filepath}"
)

Updated dataset has been saved to ./processed_data/greenfield_main_dataset.csv.
        Industry counts has been saved to ./audit_data/industry_counts.csv


Create "natural_resource" variable. Variable is True if industry is listed as a natural resource industry in the "nat_resource_list.csv" spreadsheet.

In [112]:
import pandas as pd

# Load both spreadsheets into DataFrames
main_dataset_path = "./processed_data/greenfield_main_dataset.csv"
nat_resource_list_path = "./variable_data/nat_resource_list.csv"

main_df = pd.read_csv(main_dataset_path)
nat_resource_df = pd.read_csv(nat_resource_list_path)

# Create a set of natural resource industries
natural_resource_set = set(
    nat_resource_df[nat_resource_df["natural_resource"] == True]["industry"]
)

# Create a new column "natural_resource_ind" in main_df
main_df["natural_resource_ind"] = main_df["industry"].apply(
    lambda x: x in natural_resource_set
)

# Save the modified DataFrame to CSV
main_df.to_csv(main_dataset_path, index=False)

print(f"The modified dataset has been saved to {main_dataset_path}")

The modified dataset has been saved to ./processed_data/greenfield_main_dataset.csv


Create "colonial_link" variable. Variable is True if the source country is listed in the "colonial_rulers" column for that country in "colonial_rulers.csv" and False otherwise.

In [113]:
import pandas as pd

# Load both spreadsheets into DataFrames
main_dataset_path = "./processed_data/greenfield_main_dataset.csv"
colonial_rulers_path = "./variable_data/colonial_rulers_list.csv"

main_df = pd.read_csv(main_dataset_path)
colonial_rulers_df = pd.read_csv(colonial_rulers_path)

# Create a dictionary from colonial_rulers_list
colonial_rulers_dict = colonial_rulers_df.set_index("country")[
    "colonial_rulers"
].to_dict()


# Function to check for colonial link
def has_colonial_link(row):
    destination = row["destination_country"]
    source = row["source_country"]
    colonial_rulers = colonial_rulers_dict.get(destination)

    if isinstance(colonial_rulers, str):
        colonial_rulers_list = [
            ruler.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()
            for ruler in colonial_rulers.split(", ")
        ]
        return source in colonial_rulers_list
    return False


# Apply the function to create new "colonial_link" column
main_df["colonial_link"] = main_df.apply(has_colonial_link, axis=1)

# Save the updated DatafRame to CSV
main_df.to_csv(main_dataset_path, index=False)

print(f"Updated dataset saved to {main_dataset_path}")

Updated dataset saved to ./processed_data/greenfield_main_dataset.csv


Create 2017 and 2018 spreadsheets for each individual LDC

In [114]:
import pandas as pd
import os

# Load both spreadsheets into DataFrames
main_dataset_path = "./processed_data/greenfield_main_dataset.csv"
ldcs_list_path = "./variable_data/ldcs_list_2017_2018.csv"

main_df = pd.read_csv(main_dataset_path)
ldcs_df = pd.read_csv(ldcs_list_path)


# Define function to convert country names to snake_case
def to_snake_case(s):
    return s.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()


# Convert country names in both DataFrames to snake_case
main_df["destination_country"] = main_df["destination_country"].apply(to_snake_case)
ldcs_df["country"] = ldcs_df["country"].apply(to_snake_case)

# Convert "project_date" to datetime for easier filtering
main_df["project_date"] = pd.to_datetime(main_df["project_date"], format="%b %Y")

# Extract year from "project_date"
main_df["year"] = main_df["project_date"].dt.year

# Create output directories for country spreadsheets
output_dir = "./country_spreadsheets/greenfield"

for year in [2017, 2018]:
    year_dir = os.path.join(output_dir, str(year))
    if not os.path.exists(year_dir):
        os.makedirs(year_dir)

# Create list to keep track of rows not assigned to a spreadhseet
unassigned_rows = []

# List to keep track of empty spreadsheets
empty_spreadsheets = []

# Proess each country in the ldcs_df
for country in ldcs_df["country"]:
    for year in [2017, 2018]:
        # Filter data for the specific country and year
        country_year_data = main_df[
            (main_df["destination_country"] == country) & (main_df["year"] == year)
        ]

        # Create filename
        year_dir = os.path.join(output_dir, str(year))
        filename = f"{year_dir}/{country}_greenfield_{year}.csv"

        # Save data to CSV
        if not country_year_data.empty:
            country_year_data.to_csv(filename, index=False)
        else:
            # Save an empty CSV with the same headers as greenfield_main_dataset.csv
            empty_df = main_df.head(0)
            empty_df.to_csv(filename, index=False)
            empty_spreadsheets.append(filename)

# Identify rows not assigned to any country spreadsheet
assigned_rows = main_df[main_df["destination_country"].isin(ldcs_df["country"])]
unassigned_rows = main_df[~main_df.index.isin(assigned_rows.index)]

# Save unassigned rows to a CSV file
audit_dir = "./audit_data"
unassigned_filename = f"{audit_dir}/unassigned_rows.csv"
unassigned_rows.to_csv(unassigned_filename, index=False)

# Save list of empty spreadsheets to a CSV file
empty_spreadsheets_filename = f"{audit_dir}/empty_spreadsheets.csv"
pd.DataFrame(empty_spreadsheets, columns=["empty_spreadsheets"]).to_csv(
    empty_spreadsheets_filename, index=False
)

print('Country spreadsheets created and saved to "country_spreadsheets/greenfield"')

Country spreadsheets created and saved to "country_spreadsheets/greenfield"


Create master spreadsheets for 2017 and 2018 that show:
* total_greenfield
* total_greenfield_excl_nat_res
* total_greenefield_excl_col_link
* total_greenfield_excl_qst_source
* emne_greenfield
* emne_greenfield_excl_nat_res
* emne_greenfield_excl_col_link
* emne_greenfield_excl_qst_source

In [ ]:
# import pandas as pd
# import os

# # Define the directory paths
# base_dir = "./country_spreadsheets"
# years = ["2017", "2018"]
# output_dir = "./processed_data"

# # Initialize an empty list for each year to hold data
# data_2017 = []
# data_2018 = []


# # Function to process each file and extract the required information
# def process_file(filepath, country, year):
#     df = pd.read_csv(filepath)

#     total_greenfield = df["capital_investment"].sum()
#     total_greenfield_excl_nat_res = df[df["natural_resource_ind"] == False][
#         "capital_investment"
#     ].sum()
#     total_greenfield_excl_col_link = df[df["colonial_link"] == False][
#         "capital_investment"
#     ].sum()
#     total_greenfield_excl_qst_source = df[df["source_questionable"] == False][
#         "capital_investment"
#     ].sum()
#     emne_greenfield = df[df["emne"] == True]["capital_investment"].sum()
#     emne_greenfield_exl_nat_res = df[
#         (df["emne"] == True) & (df["natural_resource_ind"] == False)
#     ]["capital_investment"].sum()
#     emne_greenfield_exl_col_link = df[
#         (df["emne"] == True) & (df["colonial_link"] == False)
#     ]["capital_investment"].sum()
#     emne_greenfield_excl_qst_source = df[
#         (df["emne"] == True) & (df["source_questionable"] == False)
#     ]["capital_investment"].sum()

#     return {
#         "country": country,
#         "year": year,
#         "total_greenfield": total_greenfield,
#         "total_greenefield_excl_nat_res": total_greenfield_excl_nat_res,
#         "total_greenfield_excl_col_link": total_greenfield_excl_col_link,
#         "total_greenfield_excl_qst_source": total_greenfield_excl_qst_source,
#         "emne_greenfield": emne_greenfield,
#         "emne_greenfield_excl_nat_res": emne_greenfield_exl_nat_res,
#         "emne_greenfield_excl_col_link": emne_greenfield_exl_col_link,
#         "emne_greenfield_excl_qst_source": emne_greenfield_excl_qst_source,
#     }


# # Loop through each year and process the files
# for year in years:
#     year_dir = os.path.join(base_dir, "greenfield", year)
#     for file_name in os.listdir(year_dir):
#         if file_name.endswith(".csv"):
#             parts = file_name.split("_")

#             # Extract country name by joining all parts except the last two
#             country = "_".join(parts[:-2])
#             filepath = os.path.join(year_dir, file_name)
#             data = process_file(filepath, country, year)
#             if year == "2017":
#                 data_2017.append(data)
#             else:
#                 data_2018.append(data)

# # Create DataFrames for 2017 and 2018
# df_2017 = pd.DataFrame(data_2017)
# df_2018 = pd.DataFrame(data_2018)

# # Sort the DataFrames
# df_2017 = df_2017.sort_values(by=["country"])
# df_2018 = df_2018.sort_values(by=["country"])

# # Save the DataFrames to CSV
# df_2017.to_csv(os.path.join(output_dir, "master_dataset_2017.csv"), index=False)
# df_2018.to_csv(os.path.join(output_dir, "master_dataset_2018.csv"), index=False)

# print("Master spreadsheets created successfully.")

Master spreadsheets created successfully.


Add governance variables and control variables, all of which can be downloaded for LDCs from World Bank DataBank here:
https://databank.worldbank.org/source/world-development-indicators

In [116]:
import pandas as pd

# Load the datasets
world_bank_data = pd.read_csv("./raw_data/world_bank_data_2016_2017.csv")
master_dataset_2017 = pd.read_csv("./processed_data/master_dataset_2017.csv")
master_dataset_2018 = pd.read_csv("./processed_data/master_dataset_2018.csv")
world_bank_var_names = pd.read_csv("./audit_data/world_bank_var_names.csv")
name_conversions = pd.read_csv("./audit_data/world_bank_name_conversions.csv")
ldcs_list = pd.read_csv("./variable_data/ldcs_list_2017_2018.csv")


# Step 1: Standardize country names
def standardize_country_names(df, conversion_df):
    # Apply name mapping
    name_map = conversion_df.set_index("old_name")["new_name"].dropna().to_dict()
    df["country"] = df["Country Name"].replace(name_map)

    # Then convert the names to snake_case
    df["country"] = (
        df["country"]
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
        .str.replace(",", "_", regex=False)
        .str.replace("'", "_", regex=False)
    )

    return df


# Replace ".." values in world_bank_data with NaN
world_bank_data.replace("..", pd.NA, inplace=True)

# Apply the name conversion process
world_bank_data = standardize_country_names(world_bank_data, name_conversions)

# Step 2: Rename variables based on world_bank_var_names.csv and reshape data
var_name_map = world_bank_var_names.set_index("old_name")["new_name"].to_dict()


def rename_world_bank_variables(df, var_map):
    df["Series Name"] = df["Series Name"].replace(var_map)
    df = df.pivot(
        index="country",
        columns="Series Name",
        values=["2016 [YR2016]", "2017 [YR2017]"],
    )
    df.columns = [
        f"{col[1]}_{col[0][:4]}" for col in df.columns.values
    ]  # Format as, e.g. "gni_per_capita_2016"
    return df.reset_index()


# Rename variables and reshape data
world_bank_data_renamed = rename_world_bank_variables(world_bank_data, var_name_map)


# Step 3: Merge transformed World Bank data with the master datasets
def merge_world_bank_data(master_df, wb_df, year_columns):
    for col in year_columns:
        # Check if the column already exists in the master dataset
        if col in master_df.columns:
            # Update existing column with values from the World Bank data
            master_df[col].update(wb_df.set_index("country")[col])
        else:
            # If the column does not exist, merge it as usual
            master_df = pd.merge(
                master_df, wb_df[["country", col]], on="country", how="left"
            )
    return master_df


# Extract relevant columns for 2016 and 2017 from world_bank_data_renamed
columns_2016 = [col for col in world_bank_data_renamed.columns if col.endswith("_2016")]
columns_2017 = [col for col in world_bank_data_renamed.columns if col.endswith("_2017")]

# Merging 2016 World Bank data with the 2017 master dataset
master_dataset_2017_with_wb = merge_world_bank_data(
    master_dataset_2017, world_bank_data_renamed, columns_2016
)

# Merging 2017 World Bank data with the 2018 master dataset
master_dataset_2018_with_wb = merge_world_bank_data(
    master_dataset_2018, world_bank_data_renamed, columns_2017
)

# Save the merged datasets
master_dataset_2017_with_wb.to_csv(
    "./processed_data/master_dataset_2017.csv", index=False
)
master_dataset_2018_with_wb.to_csv(
    "./processed_data/master_dataset_2018.csv", index=False
)


# Checking for missing data in LDCSs
def check_missing_data(ldcs, master_df):
    ldcs_df = master_df[master_df["country"].isin(ldcs["country"])]

    # Check for missing values and unstack to get missing per country/variable/year
    missing_data = ldcs_df.set_index("country").isnull()

    # Melt the DataFrame to create a record of missing data with country and year
    missing_records = missing_data.reset_index().melt(
        id_vars=["country"], var_name="variable", value_name="is_missing"
    )

    # Fitler to include only missing entries
    missing_records = missing_records[missing_records["is_missing"] == True]

    # Extract the year from the variable name (e.g., "gni_per_capita_2017" -> 2017)
    missing_records["year"] = missing_records["variable"].str.extract(r"(\d{4})")

    # Sort the data by year and country
    missing_records_sorted = missing_records.sort_values(by=["year", "country"])

    # Return the relevant columns (country, year, variable) without 'is_missing'
    return missing_records_sorted[["country", "year", "variable"]]


# Check missing data for 2017 and 2018 datasets
missing_data_2017 = check_missing_data(ldcs_list, master_dataset_2017_with_wb)
missing_data_2018 = check_missing_data(ldcs_list, master_dataset_2018_with_wb)

# Output missing data for review
print("Missing data for 2017:")
print(missing_data_2017)

print("Missing data for 2018:")
print(missing_data_2018)

Missing data for 2017:
                   country  year                   variable
517            afghanistan  2016        gni_per_capita_2016
519             bangladesh  2016        gni_per_capita_2016
531                eritrea  2016        gni_per_capita_2016
537               kiribati  2016        gni_per_capita_2016
540                liberia  2016        gni_per_capita_2016
542                 malawi  2016        gni_per_capita_2016
546                myanmar  2016        gni_per_capita_2016
550  sao_tome_and_principe  2016        gni_per_capita_2016
458           sierra_leone  2016  broadband_per_capita_2016
554                somalia  2016        gni_per_capita_2016
555            south_sudan  2016        gni_per_capita_2016
556                  sudan  2016        gni_per_capita_2016
559                 tuvalu  2016        gni_per_capita_2016
562                  yemen  2016        gni_per_capita_2016
563                 zambia  2016        gni_per_capita_2016
Missing data for 

# Adding M&A data from SDC Platinum.

Load m_and_a_dataset. Data was acquired from SDC's Mergers and Acquisitions database via WRDS. Because the acquirer ultimate parent nation (AUPNATION) variable isn't included in the WRDS version of SDC M&A, the AUPNATION variables were purchased separately, directly from LSEG/Refinitiv and added to the dataset once all non-LDC destinations were filtered out (see below).

Create a spreadsheet of all unique values appearing in "TNATION" (target nation) column in order to check against list of LDCs for different country spellings.

In [117]:
import pandas as pd

# Load datasets
input_filepath = "./raw_data/m_and_a_data_2017_2018.csv"
output_filepath = "./audit_data/m_and_a_unique_destination_countries.csv"

data = pd.read_csv(input_filepath)

# Get unique values and their counts from TNATION column
unique_values_counts = data["TNATION"].value_counts().reset_index()

# Rename the columns
unique_values_counts.columns = ["country", "count"]

# Sort df alphabetically
unique_values_counts = unique_values_counts.sort_values(by="country")
# Save the new dataframe to a CSV file
unique_values_counts.to_csv(output_filepath, index=False)

print(f"Unique values saved to {output_filepath}")

Unique values saved to ./audit_data/m_and_a_unique_destination_countries.csv


Compare list of unique destination countries with UNCTAD 2017-2018 LDCs list for data audit/validation purposes. A "name conversion" spreadsheet was created in order to convert all names of LDC countries to the same spelling/format as those used in earlier scripts above. In the next step, non-LDC destination countries ("TNATION") are filtered out of the dataset.

Filter out non-LDC destination countries from M&A data.

In [118]:
import pandas as pd

# Load the datasets
m_and_a_data = pd.read_csv("./raw_data/m_and_a_data_2017_2018.csv")
name_conversions = pd.read_csv("./audit_data/m_and_a_name_conversions.csv")
ldcs_list = pd.read_csv("./variable_data/ldcs_list_2017_2018.csv")


# Function to standardize country names and convert to snake_case
def standardize_country_names(data_df, conversion_df):
    # Apply name mapping
    name_map = conversion_df.set_index("old_name")["new_name"].dropna().to_dict()
    data_df["destination_country"] = data_df["TNATION"].replace(name_map)

    # Then convert the names to snake_case
    data_df["destination_country"] = (
        data_df["destination_country"]
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
        .str.replace(",", "_", regex=False)
        .str.replace("'", "_", regex=False)
    )

    return data_df


# Apply the name conversion function to M&A dataframe
m_and_a_data = standardize_country_names(m_and_a_data, name_conversions)

# Filter out all entries for which the target nation is not an LDC
filtered_m_and_a_data = m_and_a_data[
    m_and_a_data["destination_country"].isin(ldcs_list["country"])
]

# Identify LDC countries that do not appear in the M&A dataset
ldc_countries_set = set(ldcs_list["country"])
destination_countries_set = set(
    filtered_m_and_a_data["destination_country"].dropna().unique()
)
missing_countries = sorted(ldc_countries_set - destination_countries_set)

# Print missing countries
print(f"Number of missing LDCs: {len(missing_countries)}")
print("LDC countries not appearing in the M&A dataset's destination_country column:")
for country in missing_countries:
    print(country)

# Save the list of missing countries to a CSV file
missing_countries_filepath = "./audit_data/m_and_a_missing_ldcs.csv"
missing_countries_df = pd.DataFrame(missing_countries, columns=["missing_countries"])
missing_countries_df.to_csv(missing_countries_filepath, index=False)

# Save the filtered data
output_filepath = "./processed_data/m_and_a_main_dataset.csv"
filtered_m_and_a_data.to_csv(output_filepath, index=False)

print(f"Filtered data saved to {output_filepath}.")
print(f"Missing LDC countries saved to {missing_countries_filepath}")

Number of missing LDCs: 6
LDC countries not appearing in the M&A dataset's destination_country column:
comoros
guinea_bissau
kiribati
sao_tome_and_principe
south_sudan
tuvalu
Filtered data saved to ./processed_data/m_and_a_main_dataset.csv.
Missing LDC countries saved to ./audit_data/m_and_a_missing_ldcs.csv


LDCs not appearing in filtered dataset were manually checked against previously created list of unique TNATION values to ensure none were missed due to typos or spelling differences. LDCs listed above were manually confirmed to be absent from the M & A data.

Get list of unique countries appearing in "destination_country" and counts, as well as total size of filtered dataframe.

In [119]:
import pandas as pd

# Load datasets
m_and_a_dataset = pd.read_csv("./processed_data/m_and_a_main_dataset.csv")
ldcs_list = pd.read_csv("./variable_data/ldcs_list_2017_2018.csv")

# Get unique values appearing in "destination_country" and their counts
destination_counts = m_and_a_dataset["destination_country"].value_counts().sort_index()

print(f"Number of LDCs listed in ldcs_list_2017_2018: {len(ldcs_list)}")
print(
    f"Number of unique values appearing in 'destination_country': {len(destination_counts)}"
)
print("Unique values in 'destination_country:")
print(destination_counts)

# Get the total number of observations
total_observations = len(m_and_a_dataset)
print("\nTotal number of observations in filtered data:", total_observations)

Number of LDCs listed in ldcs_list_2017_2018: 47
Number of unique values appearing in 'destination_country': 41
Unique values in 'destination_country:
destination_country
afghanistan                      3
angola                          15
bangladesh                      23
benin                            5
bhutan                           1
burkina_faso                    14
burundi                          3
cambodia                        50
central_african_republic         2
chad                             4
democratic_republic_of_congo     9
djibouti                         4
east_timor                       2
eritrea                          1
ethiopia                        13
gambia                           1
guinea                           2
haiti                            4
laos                            13
lesotho                          7
liberia                          2
madagascar                      10
malawi                           7
mali                    

Number of unique source countries in filtered dataset (41) tallies with number of LDCs (47) minus those identified as missing (6). Dataset has been filtered down to just those transactions with LDCs as destination countries. Next step is to get ultimate parent country for all entities.

Acquirer ultimate parent nation information obtained from Refinitiv/LSEG for filtered dataset and saved as "./raw_data/sdc_m_and_a_data_incl_aupn.xlsx". Acquirer ultimate parent nation saved under variable "AUPNATION". Previous m_and_a_main_dataset.csv moved to "./archived_data/m_and_a_main_dataset.csv"

In [120]:
import shutil
import os

source_file = "./processed_data/m_and_a_main_dataset.csv"
destination_directory = "./archived_data"


def move_file(source_path, destination_dir):
    try:
        # Check if source file exists
        if not os.path.isfile(source_path):
            print(f"The file '{source_path} does not exist.")
            return

        # Check if the destination directory exists; if not, create it
        if not os.path.isdir(destination_dir):
            os.makedirs(destination_dir)
            print(f"Created directory '{destination_dir}")

        # Move the file
        shutil.move(source_path, destination_dir)
        print(f"File '{source_path}' moved to '{destination_dir}' successfully.")

    except Exception as e:
        print(f"Error moving filed: {e}")


move_file(source_file, destination_directory)

Error moving filed: Destination path './archived_data/m_and_a_main_dataset.csv' already exists


# Working with new dataset containing AUPNATION

Standardize new dataset variable names and values.

In [121]:
import pandas as pd

# Load M & A dataset
m_and_a_data = "./raw_data/sdc_m_and_a_data_incl_aupn.csv"
df = pd.read_csv(m_and_a_data)


# Define function to convert text to snake_case
def to_snake_case(s):
    return s.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()


# Convert variable names (i.e., column headers) to snake_case for data wrangling
df.columns = [to_snake_case(col) for col in df.columns]

# Create new variables "destination_country" and "source_country" containing converted values from "AUPNATION" and "TNATION" respectively.
df["destination_country"] = df["destination_country"].apply(to_snake_case)
df["source_country"] = df["aupnation"].apply(to_snake_case)

# Save modified DataFrame to new CSV file
output_filepath = "./processed_data/m_and_a_main_dataset.csv"
df.to_csv(output_filepath, index=False)

print(f"File saved to {output_filepath}.")

File saved to ./processed_data/m_and_a_main_dataset.csv.


Double check for missing LDCs.

In [122]:
import pandas as pd

# Load dataset
m_and_a_unique_set = set(
    pd.read_csv("./processed_data/m_and_a_main_dataset.csv")[
        "destination_country"
    ].unique()
)
ldcs_set = set(pd.read_csv("./variable_data/ldcs_list_2017_2018.csv")["country"])

missing_ldcs_set = sorted(ldcs_set - m_and_a_unique_set)

print(f"Number of LDCs: {len(ldcs_set)}")
print(
    f"Number of unique destination countries in new dataset: {len(m_and_a_unique_set)}"
)
print(f"Number of missing LDCs in new dataset: {len(missing_ldcs_set)}")
print(f"LDC countries not appearing in new M&A dataset destination_country column:")
for country in missing_ldcs_set:
    print(country)

Number of LDCs: 47
Number of unique destination countries in new dataset: 41
Number of missing LDCs in new dataset: 6
LDC countries not appearing in new M&A dataset destination_country column:
comoros
guinea_bissau
kiribati
sao_tome_and_principe
south_sudan
tuvalu


LDCs missing in new data that includes AUPNATION aligns with count above.

Filter out all observations except those where deal status is "complete"  and replace the previous file as "m_and_a_main_dataset.csv".

In [123]:
import pandas as pd

# Load the dataset
m_and_a_data = pd.read_csv("./processed_data/m_and_a_main_dataset.csv")

# Record the initial number of rows
initial_row_count = len(m_and_a_data)

# Filter out observations where STATUSCODE is either C or P
m_and_a_data = m_and_a_data[
    (m_and_a_data["statuscode"] == "C")
]

# Record the number of rows after filtering
filtered_row_count = len(m_and_a_data)
rows_removed = initial_row_count - filtered_row_count

# Save the filtered dataset
output_filepath = "./processed_data/m_and_a_main_dataset.csv"
m_and_a_data.to_csv(output_filepath, index=False)

print(f"Filtered dataset saved to {output_filepath}.")
print(f"Number of observations removed from dataset: {rows_removed}")
print(f"Number of rows after filtering: {filtered_row_count}")

Filtered dataset saved to ./processed_data/m_and_a_main_dataset.csv.
Number of observations removed from dataset: 103
Number of rows after filtering: 303


TODO: create a new list of missing LDCs to compare with later data.

Create a new list of missing LDCs after filtering deals.

In [124]:
import pandas as pd

# Load dataset
m_and_a_unique_set = set(
    pd.read_csv("./processed_data/m_and_a_main_dataset.csv")[
        "destination_country"
    ].unique()
)
ldcs_set = set(pd.read_csv("./variable_data/ldcs_list_2017_2018.csv")["country"])

missing_ldcs_set = sorted(ldcs_set - m_and_a_unique_set)

print(f'Number of LDCs: {len(ldcs_set)}')
print(
    f"Number of unique destination countries in new dataset: {len(m_and_a_unique_set)}"
)
print(f"Number of missing LDCs in new dataset: {len(missing_ldcs_set)}")
print(f"LDC countries not appearing in new M&A dataset destination_country column:")
for country in missing_ldcs_set:
    print(country)

Number of LDCs: 47
Number of unique destination countries in new dataset: 37
Number of missing LDCs in new dataset: 10
LDC countries not appearing in new M&A dataset destination_country column:
comoros
gambia
guinea_bissau
kiribati
liberia
sao_tome_and_principe
solomon_islands
somalia
south_sudan
tuvalu


Create spreadsheet of unique source countries (AUPNATION) for data auditing/validation.

In [125]:
import pandas as pd

# Load dataset
filepath = "./processed_data/m_and_a_main_dataset.csv"
df = pd.read_csv(filepath)

# Get unique source country names and count of each
unique_source_countries = df["source_country"].value_counts().reset_index()

# Rename columns
unique_source_countries.columns = ["country", "count"]

# Sort new df
unique_source_countries = unique_source_countries.sort_values(by="country").reset_index(
    drop=True
)

# Save to CSV
output_filepath = "./audit_data/m_and_a_unique_source_countries.csv"
unique_source_countries.to_csv(output_filepath, index=False)

print(f"File saved to {output_filepath}.")
# Display results
unique_source_countries

File saved to ./audit_data/m_and_a_unique_source_countries.csv.


,country,count
0,afghanistan,1
1,angola,1
2,australia,12
3,bahrain,1
4,bangladesh,5
...,...,...
64,united_states,19
65,unknown,9
66,vietnam,1
67,zambia,3


m_and_a_unique_source_countries.csv was used to create a m_and_a_source_country_name_conversions.csv file in order to standardize country names according to the formatting used in this replication.

Standardize source_country names according to name conversions spreadsheet.

In [126]:
import pandas as pd

# Load the datasets
m_and_a_data = pd.read_csv("./processed_data/m_and_a_main_dataset.csv")
name_conversions = pd.read_csv(
    "./audit_data/m_and_a_source_country_name_conversion.csv"
)


# Function to standardize country names and convert to snake_case
def standardize_country_names(data_df, conversion_df):
    # Apply name mapping
    name_map = conversion_df.set_index("old_name")["new_name"].dropna().to_dict()
    data_df["source_country"] = data_df["source_country"].replace(name_map)

    # Convert names to snake_case

    data_df["source_country"] = (
        data_df["source_country"]
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
        .str.replace(",", "_", regex=False)
        .str.replace("'", "_", regex=False)
    )

    return data_df


# Apply the name conversion function to the M&A DataFrame
m_and_a_data = standardize_country_names(m_and_a_data, name_conversions)

# Save the filtered data
output_filepath = "./processed_data/m_and_a_main_dataset.csv"
m_and_a_data.to_csv(output_filepath, index=False)

print(f"Standardized dataset saved to {output_filepath}.")

Standardized dataset saved to ./processed_data/m_and_a_main_dataset.csv.


Filter out all instances where source_country == unknown or where source country == destination_country (i.e., non-FDI).

In [127]:
import pandas as pd

# Load the dataset
main_dataset_filepath = "./processed_data/m_and_a_main_dataset.csv"
df = pd.read_csv(main_dataset_filepath)

# County instances where source_country == "unknown"
unknown_source_count = df[df["source_country"] == "unknown"].shape[0]

# Count instances where source_country == destination_country
non_fdi_count = df[df["source_country"] == df["destination_country"]].shape[0]

# Filter out non-FDI rows and rows where source_country == "unknown"
filtered_dataset = df[
    (df["source_country"] != df["destination_country"])
    & (df["source_country"] != "unknown")
]

# Update main dataset with filtered data

filtered_dataset.to_csv(main_dataset_filepath, index=False)

# Output results
print(f"Number of instances where source_country == unknown: {unknown_source_count}")
print(
    f"Number of instances where source_country == destination_country: {non_fdi_count}"
)
print(f"Number of rows in filtered dataset: {filtered_dataset.shape[0]}")
print(f"Filtered dataset saved as {main_dataset_filepath}")

Number of instances where source_country == unknown: 9
Number of instances where source_country == destination_country: 68
Number of rows in filtered dataset: 226
Filtered dataset saved as ./processed_data/m_and_a_main_dataset.csv


Create spreadsheet of remaning unique source countries for data auditing/validation.

In [128]:
import pandas as pd

# Load processed M&A dataset
filepath = "./processed_data/m_and_a_main_dataset.csv"
df = pd.read_csv(filepath)

# Get unique source country names and count of each
unique_source_countries = df["source_country"].value_counts().reset_index()

# Rename the columns
unique_source_countries.columns = ["country", "count"]

# Sort df
unique_source_countries = unique_source_countries.sort_values(by="country").reset_index(
    drop=True
)

# Save to CSV
output_filepath = "./audit_data/m_and_a_unique_source_countries.csv"
unique_source_countries.to_csv(output_filepath, index=False)

# Display results
unique_source_countries

,country,count
0,australia,12
1,bahrain,1
2,british_virgin_islands,1
3,cameroon,2
4,canada,10
5,cayman_islands,1
6,china,12
7,congo,1
8,cote_d_ivoire,2
9,denmark,2


Compare list of source countries with UNCTAD 2017-2018 developed country list for data audit/validation purposes.

In [129]:
import pandas as pd

# Load both spreadsheets into DataFrames
source_countries_df = pd.read_csv("./audit_data/m_and_a_unique_source_countries.csv")
developed_countries_df = pd.read_csv(
    "./variable_data/developed_countries_2017_2018.csv"
)

source_countries_set = set(source_countries_df["country"])
developed_countries_set = set(developed_countries_df["country"])

missing_in_source_countries = developed_countries_set - source_countries_set

# Display results
print(
    "Developed countries not appearing as source countries in m_and_a_main_dataset.csv:"
)
for country in sorted(missing_in_source_countries):
    print(country)

Developed countries not appearing as source countries in m_and_a_main_dataset.csv:
andorra
austria
belgium
bermuda
bulgaria
croatia
cyprus
czechia
estonia
faroe_islands
finland
gibraltar
greece
greenland
holy_see
hungary
iceland
ireland
italy
latvia
lithuania
malta
new_zealand
poland
portugal
romania
saint_pierre_and_miquelon
san_marino
slovakia
slovenia
sweden


Developed countries not appearing in m_and_a_main_dataset were checked manually against unique_source_countries.csv to ensure no matches were missed due to spelling or punctuation issues.

Compare list of countries in tax_havens.csv with countries appearing in source_country column and identify any tax_haven countries not appearing as source countries.

In [130]:
import pandas as pd

# Load both datasets as sets
source_countries_set = set(
    pd.read_csv("./audit_data/m_and_a_unique_source_countries.csv")["country"]
)
tax_havens_set = set(pd.read_csv("./variable_data/tax_havens.csv")["country"])

# Create set of missing countries
missing_tax_havens_set = tax_havens_set - source_countries_set

# Display results
print(
    "Tax haven countries not appearing as source countries in m_and_a_main_dataset.csv:"
)

for country in sorted(missing_tax_havens_set):
    print(country)

Tax haven countries not appearing as source countries in m_and_a_main_dataset.csv:
andorra
anguilla
antigua_and_barbuda
aruba
bahamas
barbados
belgium
belize
bermuda
cyprus
gibraltar
grenada
guernsey
ireland
isle_of_man
lebanon
liechtenstein
macau
malta
marshall_islands
monaco
netherlands_antilles
panama
puerto_rico
samoa
seychelles
st_kitts_and_nevis
st_lucia
st_vincent_and_grenadines
turks_and_caicos
vanuatu


Tax haven countries not appearing as source countries were manually checked against m_and_a_unique_source_countries.csv to ensure no matches were missed due to spelling or punctuation issues. No issues were found.

Create source_is_th variable. Value will be True if source_country appears in list of tax_havens and False otherwise. Tax havens list is taken from footnote 10 on page 1506 of Tørsløv, T., Wier, L., & Zucman, G. (2023). The missing profits of nations. The Review of Economic Studies, 90(3), 1499-1534.

In [131]:
import pandas as pd

# Load the datasets
m_and_a_df = pd.read_csv("./processed_data/m_and_a_main_dataset.csv")
tax_havens_df = pd.read_csv("./variable_data/tax_havens.csv")

# Create tax havens set
tax_havens_set = set(tax_havens_df["country"])

# Add "source_is_th" variable
m_and_a_df["source_is_th"] = m_and_a_df["source_country"].isin(tax_havens_set)

# Save updated DataFrame/spreadsheet
m_and_a_df.to_csv("./processed_data/m_and_a_main_dataset.csv", index=False)
print("m_and_a_main_dataset.csv updated with 'source_is_th' variable.")

# Outpute the number of observations where "source_is_th" is True to console
true_count = m_and_a_df["source_is_th"].sum()
print(f"Number of observations where 'source_is_th' is True: {true_count}")

m_and_a_main_dataset.csv updated with 'source_is_th' variable.
Number of observations where 'source_is_th' is True: 39


Create destination_is_th variable. Value will be True if destination_country appears in list of tax_havens and False otherwise. Tax havens list taken from footnote 10 on page 1506 of Tørsløv, T., Wier, L., & Zucman, G. (2023). The missing profits of nations. The Review of Economic Studies, 90(3), 1499-1534.

In [132]:
import pandas as pd

# Load the spreadsheets into DataFrames
m_and_a_df = pd.read_csv("./processed_data/m_and_a_main_dataset.csv")
tax_havens_df = pd.read_csv("./variable_data/tax_havens.csv")

# Create tax havens set
tax_havens_set = set(tax_havens_df["country"])

# Add "destination_is_th" variable
m_and_a_df["destination_is_th"] = m_and_a_df["destination_country"].isin(tax_havens_set)

# Save updated dataset
m_and_a_df.to_csv("./processed_data/m_and_a_main_dataset.csv", index=False)
print("m_and_a_df updated with 'destination_is_th' variable.")

# Output the number of observations where "destination_is_th" is True to console
true_count = m_and_a_df["destination_is_th"].sum()
print(f'Number of observations where "destination_is_th" is True: {true_count}')

# Out put unique destination_country values where "destination_is_th" is True
true_countries = m_and_a_df.loc[m_and_a_df["destination_is_th"], "destination_country"]
print("Countries where 'destination_is_th' is True:")
print(true_countries)

m_and_a_df updated with 'destination_is_th' variable.
Number of observations where "destination_is_th" is True: 2
Countries where 'destination_is_th' is True:
14     vanuatu
109    vanuatu
Name: destination_country, dtype: object


Create dataset comprising all observations where source_country is a tax haven and manually check accuracy as true origin of FDI.

In [133]:
import pandas as pd

# Load the dataset
m_and_a_df = pd.read_csv("./processed_data/m_and_a_main_dataset.csv")

# Fitler for instances where source_is_th == True
m_and_a_source_is_th_df = m_and_a_df[m_and_a_df["source_is_th"] == True]

# Select the specific columns
selected_columns = [
    "master_deal_no",
    "tmanames",
    "amanames",
    "aupnames",
    "source_country",
    "destination_country",
    "abusinessl",
    "tbusinessl",
    "atf_mid_desc",
    "ttf_mid_desc",
    "atf_macro_desc",
    "ttf_macro_desc",
]

# Retain only unique rows based on 'parent company' and selected columns
unique_parent_company_df = m_and_a_source_is_th_df[selected_columns].drop_duplicates(
    subset=["amanames"]
)

# Convert new dataframe to CSV for manual checking
unique_parent_company_df.to_csv(
    "./audit_data/m_and_a_source_is_th_unique.csv", index=False
)

num_observations = unique_parent_company_df.shape[0]

print("Filtered data saved to m_and_a_source_is_th_unique_csv")
print(f"Number of unique parent companies in filtered data: {num_observations}")

Filtered data saved to m_and_a_source_is_th_unique_csv
Number of unique parent companies in filtered data: 36


Manually check all companies to determine accuracy of parent country. Results/comments saved to "./audit_data/m_and_a_source_th_man_flagged.csv". Companies with source countries identified as tax havens were checked manually on S&P Capital IQ, Orbis, and on company websites (where available). In instances where the stated source country seemed to be either incorrect, or seemed questionable, source_country_questionable is flagged as TRUE.

Create new dataset containing only parent companies whre source country is questionable.

In [134]:
import pandas as pd

# Load the CSV file
filepath = "./audit_data/m_and_a_source_th_man_flagged.csv"
data = pd.read_csv(filepath)

# Convert the 'source_country_questionable' columh to boolean
data["source_country_questionable"] = (
    data["source_country_questionable"].str.strip().str.upper() == "TRUE"
)

# Filter rows where 'source_country_questionable' is True
filtered_data = data[data["source_country_questionable"] == True]

# Save the filtered data to a new CSV file
filtered_filepath = "./audit_data/m_and_a_source_questionable.csv"
filtered_data.to_csv(filtered_filepath, index=False)

print(f"Filtered data saved to {filtered_filepath}")

Filtered data saved to ./audit_data/m_and_a_source_questionable.csv


Use flagged data to create source_questionable in m_and_a_main_dataset.csv. Value is True if the source country has been flagged as questionable in the m_and_a_source_questionable_csv and false otherwise.

In [135]:
import pandas as pd

# Load the datasets
main_dataset_filepath = "./processed_data/m_and_a_main_dataset.csv"
questionable_source_filepath = "./audit_data/m_and_a_source_questionable.csv"

main_dataset = pd.read_csv(main_dataset_filepath)
questionable_source = pd.read_csv(questionable_source_filepath)

# Extract list of parent companies from questionable source dataset
questionable_parent_companies = questionable_source["aupnames"].unique()

# Create the "source_questionable" column in main dataset
main_dataset["source_questionable"] = main_dataset["aupnames"].isin(
    questionable_parent_companies
)

# Save the updated dataset to CSV
main_dataset.to_csv(main_dataset_filepath, index=False)

print(f"Updated datset saved to {main_dataset_filepath}")

true_count = main_dataset["source_questionable"].sum()
print(f"Number of observations with source_questionable = True: {true_count}")

Updated datset saved to ./processed_data/m_and_a_main_dataset.csv
Number of observations with source_questionable = True: 21


22 observations flagged in the M&A dataset where the source country is questionable after manual checking of all instances where "source_country" is a tax haven (see above re: list of tax haven countries).

Create "developed" variable in greenfield_main_dataset.csv. Variable takes value of True if source country appears in list of developed countries and False otherwise. Also created "emne" variable that takes the inverse value of the "developed" variable.

In [136]:
import pandas as pd

# Load spreadsheets into DataFrames

m_and_a_filepath = "./processed_data/m_and_a_main_dataset.csv"
developed_countries_filepath = "./variable_data/developed_countries_2017_2018.csv"

m_and_a_df = pd.read_csv(m_and_a_filepath)
developed_countries_df = pd.read_csv(developed_countries_filepath)

# Create set of developed countries
developed_countries_set = set(developed_countries_df["country"])

# Add "developed" variable to m_and_a_df
m_and_a_df["developed"] = m_and_a_df["source_country"].isin(developed_countries_set)

# Add "emne" variable to m_and_a_df
m_and_a_df["emne"] = ~m_and_a_df["source_country"].isin(developed_countries_set)

# Save the updated DataFrame as m_and_a_main_dataset.csv
m_and_a_df.to_csv(m_and_a_filepath, index=False)

print(f'm_and_a_main_dataset.csv updated with "developed" and "emne" variables.')

m_and_a_main_dataset.csv updated with "developed" and "emne" variables.


Create "bordering_country" variable. Variable is True if source countries shares a border with destination country and False otherwise. Bordering countries checked from CIA World Factbook https://www.cia.gov/the-world-factbook/ 

In [137]:
import pandas as pd

# Load spreadsheets into DataFrames
m_and_a_filepath = "./processed_data/m_and_a_main_dataset.csv"
bordering_countries_filepath = "./variable_data/bordering_countries_2017_2018.csv"

m_and_a_df = pd.read_csv(m_and_a_filepath)
bordering_countries_df = pd.read_csv(bordering_countries_filepath)

# Create dictionary of bordering countries
bordering_countries_dict = bordering_countries_df.set_index("ldc")[
    "bordering_countries"
].to_dict()

# Function to check if the source country is in the list of bordering countries
def is_bordering(source_country, destination_country):
    bordering_countries = bordering_countries_dict.get(destination_country, "")
    if pd.isna(bordering_countries) or not bordering_countries:
        return False
    bordering_list = bordering_countries.split(", ")
    return source_country in bordering_list

# Apply the is_bordering function to create the new "bordering_country" in m_and_a_df
m_and_a_df["bordering_country"] = m_and_a_df.apply(
    lambda row: is_bordering(row["source_country"], row["destination_country"]), axis=1
)

# Save the updated DataFrame to a new CSV file
m_and_a_df.to_csv(m_and_a_filepath, index=False)

print(
    f'The new variable "bordering_country" has been added and the updated file has been saved'
)

The new variable "bordering_country" has been added and the updated file has been saved


Create new "industry" variable by based on ttf_mid_desc variable. The Refinitiv manual for the M&A database defines ttf_mid_desc as: Target Mid Industry (Code): Refinitiv’s proprietary
mid-level industry classifications based on SIC
Codes, NAIC Codes and overall company business
description. There are more than 85 mid-level
industry classifications grouped by 14 macro-level
categories.

In [138]:
import pandas as pd

# Load processed M&A dataset
filepath = "./processed_data/m_and_a_main_dataset.csv"
df = pd.read_csv(filepath)

# Define function to convert strings to snake_case
def to_snake_case(s):
    s = (
        s.replace(" ", "_")
        .replace("'", "_")
        .replace(",", "_")
        .replace("/", "_")
        .replace("-", "_")
        .replace("(", "_")
        .replace(")", "")
        .replace(" / ", "_")
        .replace("&", "and")
    )

    return s.lower()

# Create new 'industry' column that is 'ttf_mid_desc' converted to snake_case
df["industry"] = df["ttf_mid_desc"].apply(to_snake_case)

# Save the modified DataFrame to csv
df.to_csv(filepath, index=False)

# Calculate the count of unique values in 'industry' column
industry_counts = df["industry"].value_counts().reset_index()
industry_counts.columns = ["industry", "count"]

# Sort alphabetically
industry_counts.sort_values(by="industry", inplace=True)

# Save to CSV
industry_counts_filepath = "./audit_data/m_and_industry_counts.csv"
industry_counts.to_csv(industry_counts_filepath, index=False)

print(
    f"Updated dataset has been saved to {filepath}.\n\
        Industry counts has been saved to {industry_counts_filepath}"
)


Updated dataset has been saved to ./processed_data/m_and_a_main_dataset.csv.
        Industry counts has been saved to ./audit_data/m_and_industry_counts.csv


Create natural_resource variable. Variable is True is industry is listed as a natural resource industry in the nat_resource_list.csv spreadsheet. Industries flagged as natural resource industries are:
- agriculture_and_livestock
- metals_and_mining
- oil_and_gas
- paper_and_forest_products
- tobacco


In [139]:
import pandas as pd

# Load both spreadsheets into DataFrames
main_dataset_path = "./processed_data/m_and_a_main_dataset.csv"
nat_resource_list_path = "./variable_data/m_and_a_nat_resource_list.csv"

main_df = pd.read_csv(main_dataset_path)
nat_resource_df = pd.read_csv(nat_resource_list_path)

# Create a set of natural resource industries
natural_resource_set = set(
    nat_resource_df[nat_resource_df["nat_resource"] == True]["industry"]
)

# Create a new column "natural_resource_ind" in main_df
main_df["natural_resource_ind"] = main_df["industry"].apply(
    lambda x: x in natural_resource_set
)

# Save the modified DataFrame to CSV
main_df.to_csv(main_dataset_path, index=False)

print(f"The modified dataset has been saved to {main_dataset_path}")

The modified dataset has been saved to ./processed_data/m_and_a_main_dataset.csv


Create "colonial_link" variable. Variable is True if the source country is listed in colonial_rulers.csv" and False otherwise.

In [140]:
import pandas as pd

# Load both spreadsheets into DataFrames
main_dataset_path = "./processed_data/m_and_a_main_dataset.csv"
colonial_rulers_path = "./variable_data/colonial_rulers_list.csv"

main_df = pd.read_csv(main_dataset_path)
colonial_rulers_df = pd.read_csv(colonial_rulers_path)

# Create a dictionary from colonial_rulters_list
colonial_rulers_dict = colonial_rulers_df.set_index("country")[
    "colonial_rulers"
    ].to_dict()

# Function to check for colonial link
def has_colonial_link(row):
    destination = row["destination_country"]
    source = row["source_country"]
    colonial_rulers = colonial_rulers_dict.get(destination)

    if isinstance(colonial_rulers, str):
        colonial_rulers_list = [
            ruler.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()
            for ruler in colonial_rulers.split(", ")
        ]
        return source in colonial_rulers_list
    return False

# Apply the function to create new "colonial_link" column
main_df["colonial_link"] = main_df.apply(has_colonial_link, axis=1)

# Save the updated DataFrame to CSV
main_df.to_csv(main_dataset_path, index=False)

print(f"Updated dataset saved to {main_dataset_path}")

Updated dataset saved to ./processed_data/m_and_a_main_dataset.csv


Create 2017 and 2018 spreadsheets for each individual LDC

In [141]:
import pandas as pd
import os

# Load both spreadsheets into DataFrames
main_dataset_path = "./processed_data/m_and_a_main_dataset.csv"
ldcs_list_path = "./variable_data/ldcs_list_2017_2018.csv"

main_df = pd.read_csv(main_dataset_path)
ldcs_df = pd.read_csv(ldcs_list_path)


# Define function to conver country names to snake_case
def to_snake_case(s):
    return s.strip().replace(" ", "_").replace("-", "_").replace("'", "_").lower()


# Convert country names in both DataFrames to snake_case
main_df["destination_country"] = main_df["destination_country"].apply(to_snake_case)
ldcs_df["country"] = ldcs_df["country"].apply(to_snake_case)

# Convert "dateann" to datetime for easier filtering
main_df["dateann"] = pd.to_datetime(main_df["dateann"], errors="coerce")

# Extract year from "dateann"
main_df["year"] = main_df["dateann"].dt.year

# Create output directories for country spreadsheets
output_dir = "./country_spreadsheets/m_and_a"

for year in [2017, 2018]:
    year_dir = os.path.join(output_dir, str(year))
    if not os.path.exists(year_dir):
        os.makedirs(year_dir)

# Create list to keep track of rows not assigned to a spreadsheet
unassigned_rows = []

# List to keep track of empty spreadsheets
empty_spreadsheets = []

# List to keep track of country/year pairs with no data
no_data_pairs = []

# List to keep track of countries with no data in either year
no_data_countries = []

# Process each country in the ldcs_df
for country in ldcs_df["country"]:
    country_has_data = False # Flag to track i a country has any data

    for year in [2017, 2018]:
        # Filter data for the specific country and year
        country_year_data = main_df[
            (main_df["destination_country"] == country) & (main_df["year"] == year)
        ]

        # Create filename
        filename = f"{output_dir}/{year}/{country}_m_and_a_{year}.csv"

        # Save data to CSV
        if not country_year_data.empty:
            country_year_data.to_csv(filename, index=False)
            country_has_data = True # Mark that this country has data
        else:
            # Save an empty CSV with the same headers as m_and_a_main_dataset.csv
            empty_df = main_df.head(0)
            empty_df.to_csv(filename, index=False)
            empty_spreadsheets.append(filename)
            # Track the no-data pair
            no_data_pairs.append((country, year))

    # if the country had no data in either year, add it to no_data_countries
    if not country_has_data:
        no_data_countries.append(country)

# Identify rows not assigned to any country spreadsheet
assigned_rows = main_df[main_df["destination_country"].isin(ldcs_df["country"])]
unassigned_rows = main_df[~main_df.index.isin(assigned_rows.index)]

# Save unassigned rows to a CSV file
audit_dir = "./audit_data"
if not os.path.exists(audit_dir):
    os.makedirs(audit_dir)

unassigned_filename = f"{audit_dir}/unassigned_rows_m_and_a.csv"
unassigned_rows.to_csv(unassigned_filename, index=False)

# Save the list of empty spreadsheets to a CSV file
empty_spreadsheets_filename = f"{audit_dir}/empty_spreadsheets_m_and_a.csv"
pd.DataFrame(empty_spreadsheets, columns=["empty_spreadsheets"]).to_csv(
    empty_spreadsheets_filename, index=False
)

print('Country spreadsheets created and saved to "country_spreadsheets/m_and_a"')

# Print country/year pairs with no data
if no_data_pairs:
    print("\nCountry/Year pairs with no data:")
    for country, year in no_data_pairs:
        print(f"- Country: {country}, Year: {year}")

# Print countries with no data in either year
if no_data_countries:
    print("\nCountries with no data in either year:")
    for country in no_data_countries:
        print(f"- Country: {country}")


Country spreadsheets created and saved to "country_spreadsheets/m_and_a"

Country/Year pairs with no data:
- Country: benin, Year: 2018
- Country: bhutan, Year: 2017
- Country: burundi, Year: 2017
- Country: comoros, Year: 2017
- Country: comoros, Year: 2018
- Country: djibouti, Year: 2017
- Country: east_timor, Year: 2017
- Country: east_timor, Year: 2018
- Country: eritrea, Year: 2017
- Country: gambia, Year: 2017
- Country: gambia, Year: 2018
- Country: guinea, Year: 2018
- Country: guinea_bissau, Year: 2017
- Country: guinea_bissau, Year: 2018
- Country: haiti, Year: 2017
- Country: haiti, Year: 2018
- Country: kiribati, Year: 2017
- Country: kiribati, Year: 2018
- Country: liberia, Year: 2017
- Country: liberia, Year: 2018
- Country: mauritania, Year: 2017
- Country: nepal, Year: 2017
- Country: niger, Year: 2017
- Country: sao_tome_and_principe, Year: 2017
- Country: sao_tome_and_principe, Year: 2018
- Country: sierra_leone, Year: 2018
- Country: solomon_islands, Year: 2017
- Cou

List of countries with no spreadsheets corresponds to earlier list of countries not appearing in filtered M&A dataset destination_country column (see above).

Calculate percentage of deals for which there is no deal_value given.

In [142]:
import pandas as pd

# Load the dataset
filepath = "./processed_data/m_and_a_main_dataset.csv"
data = pd.read_csv(filepath)

if "deal_value" in data.columns:
    # Number of deals with a value under "deal_value"
    deals_with_value = data["deal_value"].notnull().sum()

    # Number of deals with no value under "deal_value"
    deals_without_value = data["deal_value"].isnull().sum()

    # Percentage of deals with a value under "deal_value"
    total_deals = len(data)
    percentage_with_value = (deals_with_value / total_deals) * 100

    # Print the results
    print(f"Number of deals with a value: {deals_with_value}")
    print(f"Number of deals with no value: {deals_without_value}")
    print(f"Percentage of deals with a value: {percentage_with_value}")
else:
    print("The 'deal_value' column is missing is in the dataset.")

Number of deals with a value: 90
Number of deals with no value: 136
Percentage of deals with a value: 39.823008849557525


Add following columns to master spreadsheets for 2017 and 2018
* total_m_and_a
* total_m_and_a_excl_nat_res
* total_m_and_a_excl_col_link
* total_m_and_a_excl_qst_source
* emne_m_and_a
* emne_m_and_a_excl_nat_res
* emne_m_and_a_excl_col_link
* emne_m_and_a_excl_qst_source

In [ ]:
# import pandas as pd
# import os

# # Define the directory paths
# base_dir = "./country_spreadsheets"
# years = ["2017", "2018"]
# output_dir = "./processed_data"

# # Initialize an empty list for each year to hold data
# data_2017 = []
# data_2018 = []


# # Function to process each file and extract the required information
# def process_file(filepath, country, year):
#     df = pd.read_csv(filepath)

#     total_m_and_a = df["deal_value"].sum()
#     total_m_and_a_excl_nat_res = df[df["natural_resource_ind"] == False][
#         "deal_value"
#     ].sum()
#     total_m_and_a_excl_col_link = df[df["colonial_link"] == False]["deal_value"].sum()
#     total_m_and_a_excl_qst_source = df[df["source_questionable"] == False][
#         "deal_value"
#     ].sum()
#     emne_m_and_a = df[df["emne"] == True]["deal_value"].sum()
#     emne_m_and_a_excl_nat_res = df[
#         (df["emne"] == True) & (df["natural_resource_ind"] == False)
#     ]["deal_value"].sum()
#     emne_m_and_a_excl_col_link = df[
#         (df["emne"] == True) & (df["colonial_link"] == False)
#     ]["deal_value"].sum()
#     emne_m_and_a_excl_qst_source = df[
#         (df["emne"] == True) & (df["source_questionable"] == False)
#     ]["deal_value"].sum()

#     return {
#         "country": country,
#         "year": year,
#         "total_m_and_a": total_m_and_a,
#         "total_m_and_a_excl_nat_res": total_m_and_a_excl_nat_res,
#         "total_m_and_a_excl_col_link": total_m_and_a_excl_col_link,
#         "total_m_and_a_excl_qst_source": total_m_and_a_excl_qst_source,
#         "emne_m_and_a": emne_m_and_a,
#         "emne_m_and_a_excl_nat_res": emne_m_and_a_excl_nat_res,
#         "emne_m_and_a_excl_col_link": emne_m_and_a_excl_col_link,
#         "emne_m_and_a_excl_qst_source": emne_m_and_a_excl_qst_source,
#     }


# # Loop through each year and process the files
# for year in years:
#     year_dir = os.path.join(base_dir, "m_and_a", year)
#     for file_name in os.listdir(year_dir):
#         if file_name.endswith(".csv"):
#             parts = file_name.split("_")

#             # Extract country name by joining all parts except the last two
#             country = "_".join(parts[:-4])
#             filepath = os.path.join(year_dir, file_name)
#             data = process_file(filepath, country, year)
#             if year == "2017":
#                 data_2017.append(data)
#             else:
#                 data_2018.append(data)

# # Create DataFrames for new M&A data
# new_df_2017 = pd.DataFrame(data_2017)
# new_df_2018 = pd.DataFrame(data_2018)

# # Read the existing master spreadsheets
# master_2017_path = os.path.join(output_dir, "master_dataset_2017.csv")
# master_2018_path = os.path.join(output_dir, "master_dataset_2018.csv")

# master_df_2017 = pd.read_csv(master_2017_path)
# master_df_2018 = pd.read_csv(master_2018_path)

# # Ensure 'year' is a string in all DataFrames
# master_df_2017["year"] = master_df_2017["year"].astype(str)
# master_df_2018["year"] = master_df_2018["year"].astype(str)
# new_df_2017["year"] = new_df_2017["year"].astype(str)
# new_df_2018["year"] = new_df_2018["year"].astype(str)

# # Merge the new data with the existing DataFrames
# updated_df_2017 = master_df_2017.merge(new_df_2017, on=["country", "year"], how="left")
# updated_df_2018 = master_df_2018.merge(new_df_2018, on=["country", "year"], how="left")

# # Save the updated DataFrames back to CSV
# updated_df_2017.to_csv(master_2017_path, index=False)
# updated_df_2018.to_csv(master_2018_path, index=False)

# print("Master spreadsheets updated successfully.")

Master spreadsheets updated successfully.


Creating all greenfield and M&A variables in one script.

In [147]:
import pandas as pd
import os

# Function to process Greenfield data
def process_greenfield(filepath):
    if not os.path.exists(filepath):
        return {
            "total_greenfield": 0,
            "total_greenfield_excl_nat_res": 0,
            "total_greenfield_excl_col_link": 0,
            "total_greenfield_excl_qst_source": 0,
            "emne_greenfield": 0,
            "emne_greenfield_excl_nat_res": 0,
            "emne_greenfield_excl_col_link": 0,
            "emne_greenfield_excl_qst_source": 0,
        }

    df = pd.read_csv(filepath)
    return {
            "total_greenfield": df["capital_investment"].sum(),
            "total_greenfield_excl_nat_res": df[~df["natural_resource_ind"]]["capital_investment"].sum(),
            "total_greenfield_excl_col_link": df[~df["colonial_link"]]["capital_investment"].sum(),
            "total_greenfield_excl_qst_source": df[~df["source_questionable"]]["capital_investment"].sum(),
            "emne_greenfield": df[df["emne"]]["capital_investment"].sum(),
            "emne_greenfield_excl_nat_res": df[(df["emne"]) & (~df["natural_resource_ind"])]["capital_investment"].sum(),
            "emne_greenfield_excl_col_link": df[(df["emne"]) & (~df["colonial_link"])]["capital_investment"].sum(),
            "emne_greenfield_excl_qst_source": df[(df["emne"]) & (~df["source_questionable"])]["capital_investment"].sum(),
        
    }

# Function to process M&A data
def process_m_and_a(filepath):
    if not os.path.exists(filepath):
        return {
            "total_m_and_a": 0,
            "total_m_and_a_excl_nat_res": 0,
            "total_m_and_a_excl_col_link": 0,
            "total_m_and_a_excl_qst_source": 0,
            "emne_m_and_a": 0,
            "emne_m_and_a_excl_nat_res": 0,
            "emne_m_and_a_excl_col_link": 0,
            "emne_m_and_a_excl_qst_source": 0,
        }
    
    df = pd.read_csv(filepath)
    return {
            "total_m_and_a": df["deal_value"].sum(),
            "total_m_and_a_excl_nat_res": df[df[~"natural_resource ind"]]["deal_value"].sum(),
            "total_m_and_a_excl_col_link": df[df[~"colonial_link"]]["deal_value"].sum(),
            "total_m_and_a_excl_qst_source": df[df[~"source_questionable"]]["deal_value"].sum(),
            "emne_m_and_a": 0,
            "emne_m_and_a_excl_nat_res": 0,
            "emne_m_and_a_excl_col_link": 0,
            "emne_m_and_a_excl_qst_source": 0,
    }

Datasets created successfully.


Governance variables are already in place from previous work with greenfield data.

TODO: Dec 14th, 2024
* Check that total variables are there for all data with different controls.
* Check that proportions are there for each variable (incl. controls).
* Create dollar-value proportion variables for each greenfield variable.
* Create dollar-value proportion variables for each 'mixed' variable (greenfield + fdi)
* Create count proportion variables.

